In [6]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import collections
import sys
sys.path.append("../")
from federated.data.mitbih_data_preprocessing import get_datasets


S = 1


In [14]:
NUMBER_OF_CLIENTS=5
CLIENT_EPOCHS=10
BATCH_SIZE=32

In [15]:
train_X, train_y = get_datasets(
        train_batch_size=BATCH_SIZE,
            centralized=False,
            normalized=True,
            train_epochs=CLIENT_EPOCHS,
            number_of_clients=NUMBER_OF_CLIENTS,
            data_selector=create_unbalanced_data,
            save_data=True
            )

NameError: name 'create_unbalanced_data' is not defined

In [ ]:
@tf.function
def forward_pass(model, batch):
    predicted_y = tf.nn.softmax(tf.matmul(batch['x'], model['weights']) + model['bias'])
    return -tf.reduce_mean(tf.reduce_sum(tf.one_hot(batch['y'], 10) * tf.math.log(predicted_y), axis=[1]))
    

In [ ]:
@tff.tf_comfederated_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, lr):

    model_vars = collections.OrderedDict([
        (name, tf.Variable(name=name, initial_value=value))
        for name, value in initial_model.items()
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

    @tf.function
    def train_on_batch(model_vars, batch):
        with tf.GradientTape() as tape:
            loss = forward_pass(model_vars, batch)
        grads = tape.gradient(loss, model_vars)
        optimizer.apply_gradients(zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
        return model_vars
    return train_on_batch(model_vars, batch)

In [ ]:
@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, lr, batches)
    @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
    def batch_function(model, batch):
        return batch_train(model, batch, lr)
    return tff.sequence_reduce(batches, initial_model, batch_function)

In [ ]:
@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE, CLIENT_DATA_TYPE)
def grab_all_models(model, lr, data):
    return tff.federated_collect(tff.federated_map(
        
    ))